In [18]:
import tensorflow.keras as keras
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Lambda
from olddatasetclass import Dataset
import numpy as np
from time import time
import tensorflow as tf

from train import Args

In [2]:
def init_normal(shape=[0, 0.05], seed=None):
    mean, stddev = shape
    return keras.initializers.RandomNormal(
        mean=mean, stddev=stddev, seed=seed)


def normalize(tensor):
    K.l2_normalize(tensor)
    return tensor

In [15]:
def get_model(num_users, num_items, num_tasks, e_dim=16, f_dim=8, reg=0):
    """
    This function is used to get the Att-Mul-MF model described
    in the paper.
    Args:
        :param num_users: number of users in the dataset
        :param num_items: number of items in the dataset
        :param num_tasks: number of tasks (item genres)
        :param e_dim: the embedding dimension
        :param f_dim: the preference feature space dimension
        :param reg: regularization coefficient
    """
    # Input variables
    user_input = keras.layers.Input(shape=(1,), dtype='int32',
                                    name='user_input')
    item_input = keras.layers.Input(shape=(1,), dtype='int32',
                                    name='item_input')

    # Embedding layer
    layers = [64, 32, 16, 8] # dummy layers
    num_layer = len(layers)

    user_embedding = keras.layers.Embedding(
        input_dim=num_users, output_dim=e_dim, name='user_embedding',
        embeddings_initializer=init_normal(),
        embeddings_regularizer=keras.regularizers.l2(reg),
        input_length=1)

    item_embedding = keras.layers.Embedding(
        input_dim=num_items, output_dim=e_dim, name='item_embedding',
        embeddings_initializer=init_normal(),
        embeddings_regularizer=keras.regularizers.l2(reg),
        input_length=1)

    mlp_user_embedding = keras.layers.Embedding(
        input_dim=num_users, output_dim=int(layers[0]/2),
        name='mlp_user_embedding',
        embeddings_initializer=init_normal(),
        embeddings_regularizer=keras.regularizers.l2(reg),
        input_length=1)

    mlp_item_embedding = keras.layers.Embedding(
        input_dim=num_items, output_dim=int(layers[0]/2),
        name='mlp_item_embedding',
        embeddings_initializer=init_normal(),
        embeddings_regularizer=keras.regularizers.l2(reg),
        input_length=1)   

    # Flatten the output tensor
    user_latent = keras.layers.Flatten()(user_embedding(user_input))
    item_latent = keras.layers.Flatten()(item_embedding(item_input))
    mlp_user_latent = keras.layers.Flatten()(mlp_user_embedding(user_input))
    mlp_item_latent = keras.layers.Flatten()(mlp_item_embedding(item_input))

    # concatenate user latent and item latent, prepare for mlp part
    mlp_vector = keras.layers.Concatenate()([mlp_user_latent, mlp_item_latent])

    for idx in range(1, num_layer):
        layer = keras.layers.Dense(layers[idx],
                                   kernel_regularizer=keras.regularizers.l2(reg),
                                   activation='relu', name="mlp_layer%d" %idx)
        mlp_vector = layer(mlp_vector)
        
        
    # Element-wise product
    mf_vector = keras.layers.Multiply()([user_latent, item_latent])
    mf_vector = keras.layers.Dense(units=f_dim*num_tasks,
                                   activation='relu',
                                   kernel_initializer='lecun_uniform',
                                   name='mf_vector')(mf_vector)
    mf_vector = keras.layers.Reshape((num_tasks, f_dim))(mf_vector)
    
    weight_vector = keras.layers.Dot(axes=-1, normalize=True)([mf_vector, mlp_vector])
    att_vector = keras.layers.Dot(axes=(-1, -2))([weight_vector, mf_vector])
    
    prediction = keras.layers.Dense(1, activation='sigmoid',
                                    kernel_initializer='lecun_uniform',
                                    name='prediction')(att_vector) 
    
    # Auxiliary info output
    aux_vector = keras.layers.Dense(units=1,
                       activation='sigmoid',
                       kernel_initializer='lecun_uniform',
                       name='aux_vector')(mf_vector)
    aux_vector = keras.layers.Reshape
        
    model = keras.models.Model(inputs=[user_input, item_input],
                               outputs=[prediction, aux_vector]) ## weight_vector need to be replaced by real genre info
    return model


In [6]:
args = Args()
num_users = 6040
num_items = 3952

dataset = Dataset(args.path, args.dataset)
train, testRatings, testNegatives = dataset.train_ratings, dataset.test_ratings, dataset.negatives

testRatings_array = testRatings.iloc[:, 1].values.reshape((-1, 1))
testNegatives_array = np.asarray(testNegatives.iloc[:, 1].tolist())
testSamples = np.concatenate((testRatings_array, testNegatives_array), axis=1)

# Create user and item input samples
shape = testSamples.shape
testSamples = testSamples.reshape(-1, 1)
userSamples = np.asarray(range(shape[0])).repeat(shape[1])

In [16]:
model = get_model(num_users, num_items, args.num_tasks, args.e_dim, args.f_dim, args.reg)
t0 = time()
predictions = model.predict([userSamples, testSamples], batch_size=256)
tc = time() - t0
print(tc)

ValueError: Input 0 of layer conv2d is incompatible with the layer: expected ndim=4, found ndim=3. Full shape received: [None, 18, 8]

In [12]:
predictions[1]

array([[0.24344534, 0.3612441 , 0.6112192 , ..., 0.15881298, 0.04660348,
        0.43405923],
       [0.6326083 , 0.5067978 , 0.20544946, ..., 0.6113768 , 0.59549826,
        0.3335917 ],
       [0.        , 0.2485066 , 0.6685312 , ..., 0.14595644, 0.        ,
        0.6508298 ],
       ...,
       [0.20634267, 0.575054  , 0.3939464 , ..., 0.7528407 , 0.29209885,
        0.4768925 ],
       [0.09732835, 0.07732199, 0.42968753, ..., 0.5620707 , 0.06501652,
        0.54728657],
       [0.5082979 , 0.27702624, 0.48635542, ..., 0.22422197, 0.        ,
        0.2619219 ]], dtype=float32)

In [ ]:
def singleTaskModel(num_users, num_items, e_dim=16, f_dim=8, reg=0):
    # Input variables
    user_input = keras.layers.Input(shape=(1,), dtype='int32',
                                    name='user_input')
    item_input = keras.layers.Input(shape=(1,), dtype='int32',
                                    name='item_input')

    user_embedding = keras.layers.Embedding(
        input_dim=num_users, output_dim=e_dim, name='user_embedding',
        embeddings_initializer=init_normal(),
        embeddings_regularizer=keras.regularizers.l2(reg),
        input_length=1)

    item_embedding = keras.layers.Embedding(
        input_dim=num_items, output_dim=e_dim, name='item_embedding',
        embeddings_initializer=init_normal(),
        embeddings_regularizer=keras.regularizers.l2(reg),
        input_length=1)
    

    
    # Flatten the output tensor
    user_latent = keras.layers.Flatten()(user_embedding(user_input))
    item_latent = keras.layers.Flatten()(item_embedding(item_input))
    
    # matrix factorization
    mf_vector = keras.layers.Multiply()([user_latent, item_latent])
    out_vector = keras.layers.Dense(units=f_dim,
                                    activation='sigmoid',
                                    kernel_initializer='lecun_uniform',
                                    name='output_vector')(mf_vector)
    model = keras.models.Model(inputs=[user_input, item_input],
                               outputs=[out_vector])
    return model
    

In [ ]:
single = singleTaskModel(num_users, num_items, args.e_dim, args.f_dim, args.reg)
t0 = time()
predictions = single.predict([userSamples, testSamples], batch_size=256)
tc = time() - t0
print(tc)

In [ ]:
def dummyModel(num_users, num_items, num_tasks, e_dim=16, f_dim=8, reg=0):
    # Input variables
    user_input = keras.layers.Input(shape=(1,), dtype='int32',
                                    name='user_input')
    item_input = keras.layers.Input(shape=(1,), dtype='int32',
                                    name='item_input')

    user_embedding = keras.layers.Embedding(
        input_dim=num_users, output_dim=e_dim, name='user_embedding',
        embeddings_initializer=init_normal(),
        embeddings_regularizer=keras.regularizers.l2(reg),
        input_length=1)

    item_embedding = keras.layers.Embedding(
        input_dim=num_items, output_dim=e_dim, name='item_embedding',
        embeddings_initializer=init_normal(),
        embeddings_regularizer=keras.regularizers.l2(reg),
        input_length=1) 
    
    # Embedding layer
    layers = [64, 32, 16, 8] # dummy layers
    num_layer = len(layers)
    
    mlp_user_embedding = keras.layers.Embedding(
        input_dim=num_users, output_dim=int(layers[0]/2),
        name='mlp_user_embedding',
        embeddings_initializer=init_normal(),
        embeddings_regularizer=keras.regularizers.l2(reg),
        input_length=1)

    mlp_item_embedding = keras.layers.Embedding(
        input_dim=num_items, output_dim=int(layers[0]/2),
        name='mlp_item_embedding',
        embeddings_initializer=init_normal(),
        embeddings_regularizer=keras.regularizers.l2(reg),
        input_length=1)   
    
    # Flatten the output tensor
    user_latent = keras.layers.Flatten()(user_embedding(user_input))
    item_latent = keras.layers.Flatten()(item_embedding(item_input))
    mlp_user_latent = keras.layers.Flatten()(mlp_user_embedding(user_input))
    mlp_item_latent = keras.layers.Flatten()(mlp_item_embedding(item_input))

    
    # concatenate user latent and item latent, prepare for mlp part
    mlp_vector = keras.layers.Concatenate()([mlp_user_latent, mlp_item_latent])

    for idx in range(1, num_layer):
        layer = keras.layers.Dense(layers[idx], kernel_regularizer= keras.regularizers.l2(reg),
                                   activation='relu', name="mlp_layer%d" %idx)
        mlp_vector = layer(mlp_vector)
        
        
    # Element-wise product
    mf_vector = keras.layers.Multiply()([user_latent, item_latent])
    mf_vector = keras.layers.Dense(units=f_dim*num_tasks,
                                    activation='sigmoid',
                                    kernel_initializer='lecun_uniform',
                                    name='output_vector')(mf_vector)
    mf_vector = keras.layers.Reshape((num_tasks, f_dim))(mf_vector)
    
    weight_vector = keras.layers.Dot(axes=-1, normalize=True)([mf_vector, mlp_vector])
    att_vector = keras.layers.Dot(axes=(-1, -2))([weight_vector, mf_vector])
    
    prediction = keras.layers.Dense(1, activation='sigmoid',
                                    kernel_initializer='lecun_uniform',
                                    name='prediction')(att_vector)    
        
    model = keras.models.Model(inputs=[user_input, item_input],
                               outputs=[prediction])
    return model

In [ ]:
dummy = dummyModel(num_users, num_items, args.num_tasks, args.e_dim, args.f_dim, args.reg)
t0 = time()
predictions = dummy.predict([userSamples, testSamples], batch_size=256)
tc = time() - t0
print(tc)
# print(predictions[0].shape)
# print(predictions[1].shape)
print(predictions.shape)

In [ ]:
dummy.summary()

In [ ]:
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Layer
class MyLayer(Layer):

    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(MyLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.kernel = self.add_weight(name='kernel', 
                                      shape=(input_shape[1], self.output_dim),
                                      initializer='uniform',
                                      trainable=True)
        super(MyLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, x):
        return K.dot(x, self.kernel)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

In [27]:
a = np.array(range(24))
a = a.reshape((2,3,4))
a.shape

(2, 3, 4)

In [31]:
b = tf.constant(a, dtype=float)
b.shape

TensorShape([Dimension(2), Dimension(3), Dimension(4)])

In [32]:
dense = keras.layers.Dense(5)

In [33]:
c = dense(b)

In [34]:
c

<tf.Tensor 'dense_1/BiasAdd:0' shape=(2, 3, 5) dtype=float32>